### Домашнее задание

1. Реализовать обучение линейной регрессии для задачи boston house prices (https://www.kaggle.com/vikrishnan/boston-house-prices) с использованием torch’а

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader

from sklearn.datasets import load_boston
import pandas as pd 
import numpy as np

In [2]:
boston = load_boston()

In [3]:
data = boston['data']
target = boston['target']

In [4]:
data.shape, target.shape

((506, 13), (506,))

In [5]:
# to tensor
data  = torch.tensor(data).float()[:-10]
target = torch.tensor(target).float()[:-10]

In [6]:
X_val = torch.tensor(data).float()[-10:]
y_val = torch.tensor(target).float()[-10:]

C:\Users\BEU_RU1\AppData\Local\Temp/ipykernel_3392/3826391005.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(data).float()[-10:]
C:\Users\BEU_RU1\AppData\Local\Temp/ipykernel_3392/3826391005.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(target).float()[-10:]


#### Dataset creation

In [7]:
batch_size = 10
dataset = TensorDataset(data, target)

# Randomly reading mini-batches
data_iter = DataLoader(dataset, batch_size, shuffle=True)

# Read a batch to see how it works
for X, y in data_iter:
    print(X,y)
    break

tensor([[9.7440e-02, 0.0000e+00, 5.9600e+00, 0.0000e+00, 4.9900e-01, 5.8410e+00,
         6.1400e+01, 3.3779e+00, 5.0000e+00, 2.7900e+02, 1.9200e+01, 3.7756e+02,
         1.1410e+01],
        [2.9850e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01, 6.4300e+00,
         5.8700e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02, 1.8700e+01, 3.9412e+02,
         5.2100e+00],
        [3.6782e+00, 0.0000e+00, 1.8100e+01, 0.0000e+00, 7.7000e-01, 5.3620e+00,
         9.6200e+01, 2.1036e+00, 2.4000e+01, 6.6600e+02, 2.0200e+01, 3.8079e+02,
         1.0190e+01],
        [5.3720e-02, 0.0000e+00, 1.3920e+01, 0.0000e+00, 4.3700e-01, 6.5490e+00,
         5.1000e+01, 5.9604e+00, 4.0000e+00, 2.8900e+02, 1.6000e+01, 3.9285e+02,
         7.3900e+00],
        [3.3147e-01, 0.0000e+00, 6.2000e+00, 0.0000e+00, 5.0700e-01, 8.2470e+00,
         7.0400e+01, 3.6519e+00, 8.0000e+00, 3.0700e+02, 1.7400e+01, 3.7895e+02,
         3.9500e+00],
        [2.8392e-01, 0.0000e+00, 7.3800e+00, 0.0000e+00, 4.9300e-01, 5.7080e+00,

#### Creating model

In [8]:
model = torch.nn.Sequential(torch.nn.Linear(13,1))

In [10]:
for m in model.modules():
    if isinstance(m, torch.nn.Conv2d):
        torch.nn.init.normal(m.weight, mean=0, std=0.01)
        torch.nn.init.constant(m.bias, 0)

In [11]:
model[0].weight.data.requires_grad_(True)

tensor([[ 0.1965, -0.1939, -0.0618,  0.2262, -0.1391, -0.2367, -0.2540, -0.1979,
         -0.2470,  0.2701,  0.2229, -0.2468,  0.0509]], requires_grad=True)

In [12]:
model[0].bias.data.requires_grad_(True)

tensor([-0.1369], requires_grad=True)

In [13]:
loss = torch.nn.MSELoss(reduction='mean')

In [14]:
trainer = torch.optim.Adam(model.parameters(), lr=0.001)

#### Training

In [15]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer.zero_grad()
        l = loss(model.forward(X).reshape(-1), y)
        l.backward()
        trainer.step()
    l = loss(model.forward(data).reshape(-1), target)
    print('epoch %d, loss %f' % (epoch, l.item()))
    print('w',model[0].weight.data)
    print('b',model[0].bias.data)

epoch 1, loss 3304.497559
w tensor([[ 0.1644, -0.1579, -0.0800,  0.2412, -0.1242, -0.2051, -0.2475, -0.1562,
         -0.2787,  0.2565,  0.2464, -0.2069,  0.0416]])
b tensor([-0.1082])
epoch 2, loss 2566.875977
w tensor([[ 0.1309, -0.1255, -0.1054,  0.2605, -0.1186, -0.1815, -0.2480, -0.1218,
         -0.3160,  0.2342,  0.2607, -0.1745,  0.0234]])
b tensor([-0.0880])
epoch 3, loss 1955.856201
w tensor([[ 0.0992, -0.0942, -0.1309,  0.2770, -0.1168, -0.1616, -0.2518, -0.0900,
         -0.3502,  0.2110,  0.2713, -0.1452,  0.0051]])
b tensor([-0.0715])
epoch 4, loss 1466.477661
w tensor([[ 0.0710, -0.0655, -0.1531,  0.2917, -0.1133, -0.1414, -0.2525, -0.0601,
         -0.3810,  0.1904,  0.2826, -0.1170, -0.0127]])
b tensor([-0.0544])
epoch 5, loss 1083.588257
w tensor([[ 0.0457, -0.0403, -0.1732,  0.3083, -0.1099, -0.1225, -0.2532, -0.0332,
         -0.4101,  0.1709,  0.2928, -0.0912, -0.0277]])
b tensor([-0.0388])
epoch 6, loss 794.568420
w tensor([[ 0.0225, -0.0184, -0.1926,  0.3218, -0.

w tensor([[-0.0861,  0.0552, -0.2467,  0.8724,  0.0634,  0.2100,  0.0057, -0.0309,
         -0.5045,  0.0306,  0.3657,  0.0366, -0.3140]])
b tensor([0.1636])
epoch 47, loss 60.978855
w tensor([[-0.0860,  0.0573, -0.2466,  0.8844,  0.0676,  0.2183,  0.0094, -0.0338,
         -0.5013,  0.0300,  0.3662,  0.0361, -0.3237]])
b tensor([0.1679])
epoch 48, loss 60.903126
w tensor([[-0.0848,  0.0596, -0.2469,  0.8997,  0.0725,  0.2277,  0.0132, -0.0352,
         -0.4971,  0.0299,  0.3679,  0.0363, -0.3323]])
b tensor([0.1731])
epoch 49, loss 59.652054
w tensor([[-0.0845,  0.0612, -0.2477,  0.9134,  0.0765,  0.2363,  0.0167, -0.0393,
         -0.4937,  0.0289,  0.3677,  0.0358, -0.3441]])
b tensor([0.1771])
epoch 50, loss 60.725712
w tensor([[-0.0833,  0.0626, -0.2463,  0.9277,  0.0821,  0.2455,  0.0216, -0.0413,
         -0.4886,  0.0300,  0.3698,  0.0359, -0.3514]])
b tensor([0.1825])
epoch 51, loss 58.364376
w tensor([[-0.0843,  0.0637, -0.2477,  0.9423,  0.0852,  0.2535,  0.0232, -0.0457,
  

epoch 92, loss 44.945644
w tensor([[-0.0594,  0.0892, -0.2205,  1.5106,  0.2969,  0.6658,  0.0939, -0.1584,
         -0.2857,  0.0167,  0.4170,  0.0293, -0.6869]])
b tensor([0.4190])
epoch 93, loss 45.056442
w tensor([[-0.0610,  0.0906, -0.2198,  1.5240,  0.3018,  0.6760,  0.0934, -0.1609,
         -0.2818,  0.0157,  0.4176,  0.0279, -0.6915]])
b tensor([0.4250])
epoch 94, loss 44.553871
w tensor([[-0.0591,  0.0908, -0.2180,  1.5377,  0.3083,  0.6873,  0.0952, -0.1625,
         -0.2764,  0.0161,  0.4198,  0.0289, -0.6960]])
b tensor([0.4323])
epoch 95, loss 44.358845
w tensor([[-0.0588,  0.0889, -0.2164,  1.5500,  0.3133,  0.6974,  0.0944, -0.1667,
         -0.2705,  0.0161,  0.4207,  0.0279, -0.7011]])
b tensor([0.4382])
epoch 96, loss 44.617706
w tensor([[-0.0590,  0.0918, -0.2145,  1.5635,  0.3202,  0.7093,  0.0958, -0.1667,
         -0.2667,  0.0162,  0.4230,  0.0293, -0.7038]])
b tensor([0.4461])
epoch 97, loss 44.586361
w tensor([[-0.0584,  0.0912, -0.2128,  1.5730,  0.3258,  0.7

In [16]:
model[0].weight.data

tensor([[-0.0560,  0.0901, -0.2108,  1.6145,  0.3409,  0.7500,  0.0960, -0.1788,
         -0.2496,  0.0129,  0.4266,  0.0270, -0.7210]])

In [17]:
model[0].bias.data

tensor([0.4705])

#### Validation

In [18]:
model[0].weight.data

tensor([[-0.0560,  0.0901, -0.2108,  1.6145,  0.3409,  0.7500,  0.0960, -0.1788,
         -0.2496,  0.0129,  0.4266,  0.0270, -0.7210]])

In [19]:
for i in range (0,X_val.shape[0]):
    y_pred = model(X_val[i]).reshape(-1)
    l = y_pred - y_val[i]   
    print('y_pred %d, y_true %s, pred-true diff %f' % (y_pred.item(), y_val[i].item(), l.item()))

y_pred 19, y_true 19.100000381469727, pred-true diff 0.094286
y_pred 19, y_true 20.600000381469727, pred-true diff -1.573545
y_pred 21, y_true 15.199999809265137, pred-true diff 6.707854
y_pred 16, y_true 7.0, pred-true diff 9.785456
y_pred 11, y_true 8.100000381469727, pred-true diff 3.593592
y_pred 22, y_true 13.600000381469727, pred-true diff 9.142576
y_pred 24, y_true 20.100000381469727, pred-true diff 4.716501
y_pred 21, y_true 21.799999237060547, pred-true diff -0.331268
y_pred 19, y_true 24.5, pred-true diff -5.106640
y_pred 14, y_true 23.100000381469727, pred-true diff -8.281129


2. Реализовать наивный баесовский классификатор для MNIST (взяв всего 2 цифры “1” и “2”) сравнить с sclearn’овским

In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
%matplotlib inline
import math
import torch
import torchvision

In [28]:
digits = load_digits()

X = digits.data
y = digits.target

#Select only the digit 4 and 9 images
X = X[np.logical_or(y == 1, y == 2)]
y = y[np.logical_or(y == 1, y == 2)]

# verify selection
np.unique(y)
#array([1, 2])

# Now split them
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=200, test_size=100)

#### Sklearn

In [29]:
# Create the Multinomial Naive Bayes Classifier
clf = MultinomialNB()

In [30]:
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

# Calculate the accuracy of the prediction
print("Accuracy = {} %".format(accuracy_score(y_test, y_predicted)*100))

# Cross validate the scores
print("Classification Report \n {}".format(classification_report(y_test, y_predicted, labels=range(1,3))))

Accuracy = 94.0 %
Classification Report 
               precision    recall  f1-score   support

           1       0.95      0.91      0.93        43
           2       0.93      0.96      0.95        57

    accuracy                           0.94       100
   macro avg       0.94      0.94      0.94       100
weighted avg       0.94      0.94      0.94       100



### Pytorch

In [31]:
data_transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()])

In [32]:
mnist_train = torchvision.datasets.MNIST('.', train=True, transform=data_transform, download=True)
mnist_test  = torchvision.datasets.MNIST('.', train=False, transform=data_transform, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting .\MNIST\raw\train-images-idx3-ubyte.gz to .\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting .\MNIST\raw\train-labels-idx1-ubyte.gz to .\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting .\MNIST\raw\t10k-images-idx3-ubyte.gz to .\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting .\MNIST\raw\t10k-labels-idx1-ubyte.gz to .\MNIST\raw



In [33]:
images = torch.stack([mnist_train[i][0] for i in range(2,38) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2], 
                     dim=1).squeeze(0)
labels = torch.tensor([mnist_train[i][1] for i in range(2,38) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2])
images.shape, labels.shape

(torch.Size([10, 28, 28]), torch.Size([10]))

In [34]:
X = torch.stack([mnist_train[i][0] for i in range(len(mnist_train)) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2], 
                dim=1).squeeze(0)
Y = torch.tensor([mnist_train[i][1] for i in range(len(mnist_train)) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2])

In [35]:
n_y = torch.zeros(3)
for y in range(3):
    n_y[y] = (Y == y).sum()
P_y = n_y / n_y.sum()
P_y

tensor([0.0000, 0.5309, 0.4691])

In [36]:
n_x = torch.zeros((3, 28, 28))
for y in range(3):
    n_x[y] = torch.tensor(X.numpy()[Y.numpy() == y].sum(axis=0))
P_xy = (n_x + 1) / (n_y + 1).reshape(3, 1, 1)

In [37]:
def bayes_pred(x):
    x = x.unsqueeze(0)  # (28, 28) -> (1, 28, 28)
    p_xy = P_xy * x + (1 - P_xy)*(1 - x)
    p_xy = p_xy.reshape(3, -1).prod(dim=1)  # p(x|y)
    return p_xy * P_y

image, label = mnist_test[0]
bayes_pred(image)

tensor([0., 0., 0.])

In [38]:
a = 0.1
print('underflow:', a**784)
print('logarithm is normal:', 784*math.log(a))

underflow: 0.0
logarithm is normal: -1805.2267129073316


In [39]:
log_P_xy = torch.log(P_xy)
log_P_xy_neg = torch.log(1 - P_xy)
log_P_y = torch.log(P_y)

def bayes_pred_stable(x):
    x = x.unsqueeze(0)  # (28, 28) -> (1, 28, 28)
    p_xy = log_P_xy * x + log_P_xy_neg * (1 - x)
    p_xy = p_xy.reshape(3, -1).sum(axis=1)  # p(x|y)
    return p_xy + log_P_y

py = bayes_pred_stable(image)
py

tensor([      nan, -300.4918, -252.6362])

In [40]:
py.argmax(dim=0) == label

tensor(False)

In [41]:
def predict(X):
    return [bayes_pred_stable(x).argmax(dim=0).type(torch.int32).item() 
            for x in X]

X = torch.stack([mnist_train[i][0] for i in range(3,38) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2], dim=1).squeeze(0)
y = torch.tensor([mnist_train[i][1] for i in range(3,38) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2])
preds = predict(X)

In [42]:
X = torch.stack([mnist_train[i][0] for i in range(len(mnist_test)) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2], 
                dim=1).squeeze(0)
y = torch.tensor([mnist_train[i][1] for i in range(len(mnist_test)) if mnist_train[i][1] == 1 or mnist_train[i][1] == 2])
preds = torch.tensor(predict(X), dtype=torch.int32)
float((preds == y).sum()) / len(y)  # Validation accuracy

0.09584513692162418